In [1]:
import pandas as pd
import numpy as np
import math as math
from matplotlib import pyplot as plt
from IPython.display import display, Math, HTML
import scipy.optimize as op
from pylab import *
!pip install tabulate colorama
from math import exp,log,fabs
import pandas as pd
from tabulate import tabulate
from colorama import Fore, Style
import matplotlib.pyplot as plt

Se usa un evaporador de efecto triple y de alimentación hacia adelante para evaporar una disolución de azúcar que contiene 10% de sólidos en peso, hasta una concentración de 50% en peso. La Elevación del Punto de Ebullición (EPE) de las soluciones (independiente de la presión) puede estimarse con la expresión:
\begin{equation}
EPE (°C)=1.78x+6.22x^{2}
\end{equation}
\begin{equation}
EPE (°F)=3.2x+11.2x^{2}
\end{equation}
donde x es la fracción en peso de azúcar en solución (K1)= Se usa vapor de agua saturado a 205.5 kPa (29.8 lb/pulg2 abs). La presión en el espacio del vapor del tercer efecto es 13.4 kPa (1.94 lb/pulg2 abs). La velocidad de alimentación es 22 680 kg/h (50 000 lbm/h) a 26.7 °C (80°F). La capacidad calorífica de la disolución líquida es (KI)
\begin{equation}
Cp=4.19-2.35x (kJ/kg K)
\end{equation}

\begin{equation}
Cp=1.0-0.56x (btu/lbm °F)
\end{equation} . Se considera que el calor de disolución es despreciable. La estimación de los coeficientes de transferencia de calor arroja los siguientes resultados: U1= 3123, U2= 1987 y U3= 1136 W/m2 K o 550, 350 y 200 btu/h ft2 °F. Suponiendo la misma área superficial en los efectos, calcule el área, la cantidad de vapor de agua.

In [4]:
U1=3123
U2=1987
U3=1136
def EPE(x):
  return (1.78*x)+(6.22*x**2)

def CP(x):
  return 4.19-(2.35*x)



#Calcule la temperatura de saturación del vapor en el 3er efecto
Ts4=float(input("Inserte la temperatura de saturación del vapor en el 3er efecto (°C):"))
#Inserte la temperatura de saturación en la primer chaqueta
Ts1=float(input("Inserte la temperatura de saturación del vapor en la 1er chaqueta (°C):"))
xL=0.5
xF=0.1
F=22680
L3=(F*xF)/xL #Se determina la cantidad de solución de salida en el 3er efecto
V=F-L3 #Se determina la cantidad de vapor producida en todo el evaporador de triple efecto
V1=V/3 #Se supone que V se divide en parte iguales por cada evaporador.
V2=V/3
V3=V/3
T0=26.7
#Por medio de ecuaciones de balance determinamos el flujo de líquido a cada evaporador

L1=F-V1
L2=L1-V2
L3=L2-V3
N=1

x1=(F*xF)/L1 #Por medio de balance, determinamos la fracción de soluto en cada evaporador
x2=(L1*x1)/L2
x3=(L2*x2)/L3

EPE1=EPE(x1) #Se calcula la elevación al punto de ebullición por evaporador
EPE2=EPE(x2)
EPE3=EPE(x3)

DeltaT=Ts1-Ts4-EPE1-EPE2-EPE3 #Se calcula la Delta T

def DT(U):
  return DeltaT*((1/U)/((1/U1)+(1/U2)+(1/U3)))

DT1=DT(U1) #Se calcula una primer Delta T en el evaporador 1
DT2=DT(U2)
DT3=DT(U3)

T1=Ts1-DT1 #Se calcula la temperatura de ebullición en el evaporador 1
T2=T1-EPE1-DT2
Ts2=T1-EPE1 #Se calcula la temperatura de saturación del vapor a la salida del evaporador 2
T3=T2-EPE2-DT3
Ts3=T2-EPE2




while N==1:

  x1=(F*xF)/L1
  x2=(L1*x1)/L2
  x3=(L2*x2)/L3

  EPE1=EPE(x1)
  EPE2=EPE(x2)
  EPE3=EPE(x3)



  T1=Ts1-DT1
  T2=T1-EPE1-DT2
  Ts2=T1-EPE1
  T3=T2-EPE2-DT3
  Ts3=T2-EPE2

  Tipo=["T (Sat)","Temp. Evap"]
  TT1=[Ts1,T1]
  TT2=[Ts2,T2]
  TT3=[Ts3,T3]
  TC=[Ts4,0]

  data = {
    "Tipo": Tipo,
    "Efecto 1 (°C)": TT1,
    "Efecto 2 (°C)": TT2,
    "Efecto 3 (°C)": TT3,
    "Condensador (°C)":TC,


  }

  df = pd.DataFrame(data)

  table = tabulate(df, headers='keys', tablefmt='grid')

  colored_table = ""
  for line in table.split('\n'):
      colored_line = ""
      for item in line.split('│'):
          if item.strip().replace('.', '', 1).isdigit():
              value = float(item.strip().replace('│', ''))
              if value < 0.5:
                  colored_line += Fore.RED + item + Style.RESET_ALL + '│'
              else:
                  colored_line += Fore.GREEN + item + Style.RESET_ALL + '│'
          else:
              colored_line += item + '│'
      colored_table += colored_line + '\n'

  print("Tabla de composiciones con colores:")
  print(colored_table)

  Fcp=CP(xF)
  L1cp=CP(x1)
  L2cp=CP(x2)
  L3cp=CP(x3)

  H1=float(input("Inserte la entalpía del vapor sobrecalentado a T1 y Psat2 (kJ/kg): ")) #Se inserta las entalpías para realizar un balance de energía
  lambda1=float(input("Inserte el calor latente de vaporización a Psat1 (kJ/kg):"))
  H2=float(input("Inserte la entalpía del vapor sobrecalentado a T2 y Psat3 (kJ/kg): "))
  lambda2=H1-float(input("Inserte la entalpía de líquido saturado a Tsat2 (kJ/kg):"))
  H3=float(input("Inserte la entalpía de vapor sobrecalentado a T3 y Psat del condensador (kJ/kg):")) #Se inserta la entalpía del líquido saturado para calcular los calores latentes
  lambda3=H2-float(input("Inserte la entalpía de líquido satuado a Tsat3 (kJ/kg):"))


  # Definir la matriz de coeficientes A
  A = np.array([
       #S         L1      L2  V1  V2  V3
      [lambda1, -L1cp*T1, 0, -H1, 0, 0], #Se propone un sistema de ecuaciones de balance de energía
      [0, -1, 0, -1, 0, 0],
      [0, 1, -1, 0, -1, 0],
      [0, 0, 1, 0, 0, -1],
      [0, L1cp*T1, -L2cp*T2, lambda2, -H2, 0],
      [0, 0, L2cp*T2, 0, lambda3, -H3]
  ])

  # Definir el vector de términos independientes B
  B = np.array([-F*Fcp*T0, -F, 0, L3, 0, L3cp*T3])

  # Resolver el sistema de ecuaciones
  X = np.linalg.solve(A, B)



  data = {
    "S (kg/g)": [X[0]],
    "L1 (kg/h)": [X[1]],
    "L2 (kg/h)": [X[2]],
    "L3 (kg/h)": [L3],
    "V1 (kg/h)": [X[3]],
    "V2 (kg/h)": [X[4]],
    "V3 (kg/h)": [X[5]],


  }

  df = pd.DataFrame(data)

  table = tabulate(df, headers='keys', tablefmt='grid')

  colored_table = ""
  for line in table.split('\n'):
      colored_line = ""
      for item in line.split('│'):
          if item.strip().replace('.', '', 1).isdigit():
              value = float(item.strip().replace('│', ''))
              if value < 0.5:
                  colored_line += Fore.RED + item + Style.RESET_ALL + '│'
              else:
                  colored_line += Fore.GREEN + item + Style.RESET_ALL + '│'
          else:
              colored_line += item + '│'
      colored_table += colored_line + '\n'

  print("Tabla de composiciones con colores:")
  print(colored_table)

  A1=(X[0]*lambda1)/(U1*DT1*3.6) #Se calculan las áreas
  A2=(X[3]*lambda2)/(U2*DT2*3.6)
  A3=(X[4]*lambda3)/(U3*DT3*3.6)

  data = {
    "A1 (m2)": [A1],
    "A2 (m2)": [A2],
    "A3 (m2)": [A3],



  }

  df = pd.DataFrame(data)

  table = tabulate(df, headers='keys', tablefmt='grid')

  colored_table = ""
  for line in table.split('\n'):
      colored_line = ""
      for item in line.split('│'):
          if item.strip().replace('.', '', 1).isdigit():
              value = float(item.strip().replace('│', ''))
              if value < 0.5:
                  colored_line += Fore.RED + item + Style.RESET_ALL + '│'
              else:
                  colored_line += Fore.GREEN + item + Style.RESET_ALL + '│'
          else:
              colored_line += item + '│'
      colored_table += colored_line + '\n'

  print("Tabla de composiciones con colores:")
  print(colored_table)
  print("Si la similitud de las áreas es lo suficientemente parecida teclee 0, de lo contrario oprima 1") #Si las áreas no son lo suficientemente parecidas se continúa la iteración
  N=float(input("Inserte 0 (para finalizar) o 1 (para continuar la iteración)"))
  L1=X[1]
  L2=X[2]
  A=(((X[0]*lambda1)/(U1*3.6))+((X[3]*lambda2)/(U2*3.6))+((X[4]*lambda3)/(U3*3.6)))/DeltaT #Se propone una nueva área para forzar a las deltas de temperatura a acercarse a algún área en específico
  DT1=(X[0]*lambda1)/(U1*A*3.6)
  DT2=(X[3]*lambda2)/(U2*A*3.6)
  DT3=(X[4]*lambda3)/(U3*A*3.6)





Inserte la temperatura de saturación del vapor en el 3er efecto (°C):51.65
Inserte la temperatura de saturación del vapor en la 1er chaqueta (°C):121.071
Tabla de composiciones con colores:
+----+------------+-----------------+-----------------+-----------------+--------------------+│
|    | Tipo       |   Efecto 1 (°C) |   Efecto 2 (°C) |   Efecto 3 (°C) |   Condensador (°C) |│
+====+============+=================+=================+=================+====================+│
|  0 | T (Sat)    |         121.071 |        108.318  |         88.1697 |              51.65 |│
+----+------------+-----------------+-----------------+-----------------+--------------------+│
|  1 | Temp. Evap |         108.676 |         88.8368 |         54.095  |               0    |│
+----+------------+-----------------+-----------------+-----------------+--------------------+│

Inserte la entalpía del vapor sobrecalentado a T1 y Psat2 (kJ/kg): 2689.26
Inserte el calor latente de vaporización a Psat1 (kJ/kg):2199.